In [1]:
# importing modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from modules.library import documentRetrieval as dr
from modules.library import queryExp as qe
from modules.library.postgresql import PostgresQL
pg = PostgresQL() 


C:\Users\sarab\Miniconda3\envs\envirolens\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# importing word vectors
from gensim.models import KeyedVectors
wiki_en_align = './../data/fasttext/wiki.en.align.vec' #'../../data/fasttext/wiki.en.align.vec'
# get fasttext wiki embeddings for english
wv_wiki_en = KeyedVectors.load_word2vec_format(wiki_en_align)
print('english words {}'.format(len(list(wv_wiki_en.vocab.keys()))))

C:\Users\sarab\Miniconda3\envs\envirolens\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


english words 2519370


In [3]:
# prepare stopword list
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
stopWords = stopwords.words('english')

In [4]:

pg.connect(database="eurlex_environment_only", user="postgres", password="dbpass") #"eurlex_env_only" "solata.2018"

In [5]:
def db_query(query_words):
    """ Iz baze vrne seznam slovarjev, ki vsebujejo dokumente (tekst) in njihove IDje, ki v tekstu vsebujejo vsaj eno od besed iz seznama(list) query_words"""
    output = '|'.join(query_words)
    SQL = """
            SELECT document_id, document_text FROM documents
            WHERE document_text @@ to_tsquery(""" + '\''+ output + '\'' + """);"""
    documents = pg.execute(SQL)
    return(documents)

In [6]:
tokenized = qe.tokenize('water pollution underground', stopWords)
ext = qe.extend_tokens(tokenized, wv_wiki_en)
candidates = qe.pre_retrieval_KNN('water pollution underground', 5, wv_wiki_en, 10, stopWords,extension=False)

In [7]:
docs = db_query(tokenized) 
#premisli kaj (tokens, ext, candidates) das v db_query (I guess vse? --> ce dela na osnovnem queryju dovolj dobro pusti osnovni, drugace prevec dokumentov)

In [ ]:
#print(len(docs))

In [ ]:
#docs[0]

In [ ]:
#ce vzame cas popravi v metriki
def change_dict_structure(dict_list):
    """Takes list of dicts from db_query and changes to dict with key=id, value = text (used for metrices)."""
    texts = {}
    for dic in dict_list:
        doc_id = dic.get('document_id')
        text = dic.get('document_text')
        texts.update({doc_id: text})
    return texts

In [ ]:
texts_dict = change_dict_structure(docs)

In [41]:
#texts_dict.get(2)

In [ ]:
multiply0 = dr.probability_score(tokenized,texts_dict, probability_multiply,10)
multiply1 = dr.probability_score(tokenized+ext,texts_dict, probability_multiply,10)
multiply2 = dr.probability_score(tokenized+ext+candidates,texts_dict, probability_multiply,10)
# takes also probability_function probability_sum_weight, but doesnt give final result (used in probability_score_sum_weights)
    # top_expansion are the onesthat are not counted as original query, usually ext ((extension by summation of 2 consecutive words) counts as original query
#     """Assigns score to documents based on probability_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words and/or KNN candidates).
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: probability_multiply, probability_sum. Require only first 4 arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion), then have to be also in tokens.
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """


In [ ]:
dr.probability_score_sum_weights(original_tokens, top_expansion, texts,n, alpha, wv)
# """As probability_score only weighted.
#         Args:
#         original_tokens(list): List of strings. Tokenized original query. Usually also extension (extension by summation of 2 consecutive words), but not top_expansion (kNN candidates)
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         texts (dict):  Keys represent document ids, values are document text.
#         n (int): Number of returned tuples, sorted by highest scores.
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_score (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """

In [ ]:
dr.tfidf_score(tokens, texts, tfidf_function,n, *args)
#     """Assigns score to documents based on tfidf_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words).For tfidf_sum metric can also add kNN candidates.
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: tfidf_sum; require only first 4 arguments, tfidf_sum_weight; require all arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#         not_appear (list): List of words that did not occure in any document.
#     """

In [22]:
import json
import xml.etree.ElementTree as ElementTree

def sgml_to_json(file):
    with open(file, 'r') as f:   # Reading file
        xml = f.read()

    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = []
    for doc in root:
        doc_json = {'DOCNO: {}, TEXT: {}'.format( # Nice formatting py 3 \o/
                doc.find('DOCNO').text.strip(),
                doc.find('TEXT').text.strip(),
            )
            }
        docs_json.append(doc_json)
    return docs_json
    

In [12]:
import os
import json
import xml.etree.ElementTree as ElementTree

directory = '/Users/sarab/work/try/TREC/TREC-Disk-4/FT/FT911'
directory_list = []
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-4\\FT\\FT911\\'+filename, 'r') as f:   # Reading file
        xml = f.read()

    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = []
    for doc in root:
        doc_json = {'DOCNO: {}, TEXT: {}'.format( # Nice formatting py 3 \o/
                doc.find('DOCNO').text.strip(),
                doc.find('TEXT').text.strip(),
            )
            }
        docs_json.append(doc_json)
    directory_list = directory_list + docs_json
    print(filename)
    continue
  


FT911_1
FT911_10
FT911_11
FT911_12
FT911_13
FT911_14
FT911_15
FT911_2
FT911_3
FT911_4
FT911_5
FT911_6
FT911_7
FT911_8
FT911_9


In [11]:
docs_json #ni vseh datotek, ugotovi kaj se dogaja, 



[{"DOCNO: FT911-2950, TEXT: THE POLITICAL issue dominating the Eden District Council elections this year\nis whether party politics are, in fact, desirable at all. Bounded by the\nLake District to the west and the Pennines to the east, the 900 square mile\nValley of Eden has long been an earthly paradise with sheep as plentiful as\nideologues are rare.\nBut are they any more? In the George Hotel amd the conspiratorial coffee\nhouses of Penrith's Devonshire Street, citizens are agog over the Cumberland\nand Westmorland Herald's sensational headline: 'Eden Election Takes On a\nPolitical Outlook.'\nAt issue is the revelation that of the council 37 seats an unprecedented 17\nare being contested. Worse still, in Penrith itself - Eden's capital with\nsome 13,000 of the Valley's 47,000 population - conventional parties have\nswept in to challenge a long tradition of dogma-free politics.\nThe outgoing council had just two non-independent councillors. Now all three\nmain parties have set up the

In [15]:
doc_json

{"DOCNO: doc.find('DOCNO').text.strip(), TEXT: doc.find('TEXT').text.strip()"}

In [12]:
ab = {"cow": "milk"}

In [13]:
ab


{'cow': 'milk'}